In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.datasets import make_blobs
import random
import json
import time
from tabulate import tabulate

from kdtree import *
from helper import *

In [2]:
savefile = "mst_data2.json"
to_plot = False
no_points = 150
no_centres = 3

In [3]:
# from sklearn.datasets import make_blobs

# X, Y = make_blobs(n_samples=no_points, centers=no_centres, random_state=42)
# points = [(x, y) for x, y in X]

In [4]:
from sklearn.datasets import load_iris
from sklearn.mixture import GaussianMixture

iris = load_iris()
X_original = iris.data
y_original = iris.target

gmm = GaussianMixture(n_components=3, random_state=42) 
gmm.fit(X_original)

X_synthetic, y_synthetic = gmm.sample(n_samples=10000)

points = [(x[0], x[1]) for x in X_synthetic[:, :2] ]
points = list(set(points))


no_points = len(points)


In [5]:
print(len(points))

10000


In [6]:
# from sklearn.datasets import make_blobs, make_moons , make_circles



# # For moons
# no_points_moons = 100  # example value
# X_moons, Y_moons = make_moons(n_samples=no_points, noise=0.05, random_state=42)
# points = [(x, y) for x, y in X_moons]
# X, Y = make_circles(n_samples=no_points, factor=0.15, noise=0.05)
# points = [(x, y) for x, y in X]


In [7]:
# import numpy as np
# from sklearn.datasets import make_moons
# import matplotlib.pyplot as plt

# def shift_moons(points, shift):
#     return points + shift

# # Generate the first set of moons
# X1, y1 = make_moons(n_samples=70, noise=0.05, random_state=42)

# # Generate and shift the second set of moons
# X2, y2 = make_moons(n_samples=80, noise=0.05, random_state=42)
# X2 = shift_moons(X2, np.array([1.5, 1]))  # Example shift

# # Combine datasets
# X = np.concatenate([X1, X2])
# y = np.concatenate([y1, y2 + 2])  # Update labels for the second moon
# points = [(x, y) for x, y in X]



In [8]:
cordmap = {point: i for i, point in enumerate(points)}
G = nx.Graph()
for coord, index in cordmap.items():
    G.add_node(index, pos=coord)
graphify(G, to_plot , bottom_text = f'blobs no of points = {no_points},')


In [9]:
def dcran(cordmap):
    tree = KDTree()
    tree.root = tree.build(points)
    G = nx.Graph()
    for coord, index in cordmap.items():
        G.add_node(index, pos=coord)

    k = 2
    connected_components = 1
    while True:
        # graphify(G, to_plot)
        maxdis = min(len(cordmap) , k**k)
        for pointi, i in cordmap.items():
            pointj = ith_nearest_neighbor(tree, pointi, k)
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, cordmap[pointj], weight=dis)
        graphify(G, to_plot)
            # print(k ,  i , cordmap[pointj] , dis )
            # randis = random.randint(k + 1, maxdis)
        for pointi, i in cordmap.items():
            randis = maxdis
            # randis = (cusrandis(k + 1 , maxdis))
            # print(randis , k)
            pointj = ith_nearest_neighbor(tree, pointi, randis)
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, cordmap[pointj], weight=dis)
            # print(k ,  i , cordmap[pointj] , dis , randis)
        
        graphify(G, to_plot)
        ccount = count_clusters(G)
        print(k, ccount)
        k += 1
        
        if ccount == 1:
            break
        
    return G

In [10]:
dcran_start_time = time.time()
kc = dcran(cordmap)

2 90
3 1


In [11]:
graphify(kc, to_plot, bottom_text = f'after edge selection , no of edges = {kc.number_of_edges()}')

In [12]:
G = kc
num_nodes = G.number_of_nodes()
conedge = num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")

Total number of nodes: 10000
Total number of edges: 28445
Total sum of edge weights: 1021.9701273661574


In [13]:
mst = nx.minimum_spanning_tree(G, algorithm="prim", weight="weight")
graphify(mst, to_plot , bottom_text = f'iris using approximation algorithm')



In [14]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()


foundw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
dcran_end_time = time.time()
dcran_elapsed_time = dcran_end_time - dcran_start_time

Total number of nodes: 10000
Total number of edges: 9999
Total sum of edge weights: 157.20512577196067


In [15]:
stmst_start_time = time.time()
G = nx.Graph()

for pointi, i in cordmap.items():
    G.add_node(i, pos=pointi)
    for pointj, j in cordmap.items():
        if i != j:
            dis = euclidean_distance(pointi, pointj)
            G.add_edge(i, j, weight=dis)

In [16]:
graphify(G, to_plot , bottom_text = f'iris using prims')

In [17]:
mst = nx.minimum_spanning_tree(G, algorithm="prim", weight="weight")
graphify(mst, to_plot , bottom_text = f'iris using prims')

In [18]:
G = mst
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

# Calculate the sum of all edge weights
realw = total_weight = sum(data["weight"] for u, v, data in G.edges(data=True))

print(f"Total number of nodes: {num_nodes}")
print(f"Total number of edges: {num_edges}")
print(f"Total sum of edge weights: {total_weight}")
stmst_end_time = time.time()
stmst_elapsed_time = stmst_end_time - stmst_start_time

Total number of nodes: 10000
Total number of edges: 9999
Total sum of edge weights: 155.2697442302945


In [19]:
percentage_error = ((foundw - realw) / realw) * 100

# Format the output for readability
formatted_output = f"Real Weight: {realw}  Found Weight: {foundw}  no of Edge: {conedge} Percentage Error: {percentage_error:.2f}%"
formatted_output

'Real Weight: 155.2697442302945  Found Weight: 157.20512577196067  no of Edge: 28445 Percentage Error: 1.25%'

In [20]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)
currres = []
speedup = round((stmst_elapsed_time  / dcran_elapsed_time) , 2)
loaded_data.append(
    [
        no_points,
        no_centres,
        foundw,
        realw,
        conedge,
        100 - percentage_error ,
        dcran_elapsed_time ,
        stmst_elapsed_time ,
        speedup
    ]
)

# Save the updated dictionary back to the JSON file
with open(savefile, "w") as f:
    json.dump(loaded_data, f)

[[1000, 10, 262.4419314705513, 258.819134723451, 8252, 98.6, 1.83, 4.38, 2.4], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.921, 28.58, 551.35, 19.29], [10000, 100, 1342.6324139702915, 1337.4478711909765, 27591, 99.612, 3.24, 510.53, 157.39], [10000, 100, 1342.6324139702915, 1337.4478711909762, 27591, 99.612, 2.48, 632.89, 255.22], [10000, 1000, 1459.7289235192638, 1455.1233116849917, 27457, 99.683, 2.5, 565.44, 226.17], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.179672241210938, 617.7955191135406, 25.55], [10000, 50, 1260.1226131964793, 1255.8337705131123, 27661, 99.65848643474409, 2.1823902130126953, 513.3643357753754, 235.23], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.11609435081482, 543.8352909088135, 22.55], [10000, 10, 875.7172119877604, 865.3329904388614, 38587, 98.79997392175785, 16.157432556152344, 646.0387144088745, 39.98], [10000, 100, 1342.6324139702872, 1337.4478711909717, 27591, 99.612

In [21]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)

[[1000, 10, 262.4419314705513, 258.819134723451, 8252, 98.6, 1.83, 4.38, 2.4], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.921, 28.58, 551.35, 19.29], [10000, 100, 1342.6324139702915, 1337.4478711909765, 27591, 99.612, 3.24, 510.53, 157.39], [10000, 100, 1342.6324139702915, 1337.4478711909762, 27591, 99.612, 2.48, 632.89, 255.22], [10000, 1000, 1459.7289235192638, 1455.1233116849917, 27457, 99.683, 2.5, 565.44, 226.17], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.179672241210938, 617.7955191135406, 25.55], [10000, 50, 1260.1226131964793, 1255.8337705131123, 27661, 99.65848643474409, 2.1823902130126953, 513.3643357753754, 235.23], [10000, 10, 866.0209239473647, 865.3329904388614, 90645, 99.92050071867081, 24.11609435081482, 543.8352909088135, 22.55], [10000, 10, 875.7172119877604, 865.3329904388614, 38587, 98.79997392175785, 16.157432556152344, 646.0387144088745, 39.98], [10000, 100, 1342.6324139702872, 1337.4478711909717, 27591, 99.612

In [22]:
headers = [
    "Points",
    "Centres",
    "DCRAN Wt",
    "prims Wt",
    "Edge count",
    "Acc(%)",
    "DCRAN Time (s)",
    "STMST Time (s)",
    "Speedup"
]

# Format the data as a table using tabulate
table_str = tabulate(loaded_data[-15:], headers, tablefmt="pipe", floatfmt=(".0f", ".0f", ".1f", ".1f", ".0f", ".2f", ".2f", ".2f", ".2f"))
print(table_str)




|   Points |   Centres |   DCRAN Wt |   prims Wt |   Edge count |   Acc(%) |   DCRAN Time (s) |   STMST Time (s) |   Speedup |
|---------:|----------:|-----------:|-----------:|-------------:|---------:|-----------------:|-----------------:|----------:|
|     1000 |         2 |      137.3 |      134.4 |         6063 |    97.82 |             2.17 |             4.03 |      1.86 |
|     1000 |         1 |       96.5 |       94.8 |         2147 |    98.26 |             0.18 |             3.75 |     21.09 |
|     5000 |         2 |      320.6 |      313.8 |        37775 |    97.86 |            53.53 |           126.53 |      2.36 |
|     1000 |       100 |      426.8 |      420.4 |         2162 |    98.50 |             0.19 |             2.48 |     12.77 |
|     1000 |       100 |      426.8 |      420.4 |         2162 |    98.50 |             0.20 |             2.91 |     14.77 |
|      100 |        10 |       97.4 |       92.6 |          367 |    94.75 |             1.43 |             0.2

In [23]:
print("Line 1\nLine 2")


Line 1
Line 2
